# AWS SageMaker Resource Cleanup

**Project:** AAI-540 Machine Learning Operations - Final Team Project  
**Context:** Cleanup after notebook 06 - Model Deployment  
**Objective:** Delete SageMaker endpoints, models, and associated resources to avoid ongoing charges

---

## Table of Contents
1. [Setup & Configuration](#setup)
2. [Load Deployment Info](#load)
3. [Delete Endpoint](#endpoint)
4. [Delete Model Resources](#model)
5. [Delete Model Registry Entries](#registry)
6. [Delete S3 Artifacts (Optional)](#s3)
7. [Cleanup Summary](#summary)

---

**WARNING:** This notebook deletes AWS resources. Review each section before running to ensure you want to delete these resources.

## 1. Setup & Configuration

Load required libraries and configure AWS clients.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import json
import boto3
import sagemaker
from datetime import datetime
from pathlib import Path

# Initialize SageMaker session and clients
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
bucket = sagemaker_session.default_bucket()

sagemaker_client = boto3.client('sagemaker')
s3_client = boto3.client('s3')

print(f"Region: {region}")
print(f"Bucket: {bucket}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Region: us-east-1
Bucket: sagemaker-us-east-1-864106638709


## 2. Load Deployment Info

Load deployment configuration from notebook 06 or use stored variables.

In [2]:
# Try to load from stored variables first
try:
    %store -r endpoint_name
    %store -r model_package_arn
    %store -r deployment_config
    print(f"Loaded from store: {endpoint_name}")
except:
    # Fallback to deployment config file
    try:
        with open('deployment_config.json', 'r') as f:
            deployment_config = json.load(f)
        endpoint_name = deployment_config['endpoint_name']
        model_package_arn = deployment_config.get('model_package_arn')
        print(f"Loaded from file: {endpoint_name}")
    except FileNotFoundError:
        print("ERROR: No deployment configuration found.")
        print("Please set endpoint_name manually or run notebook 06 first.")
        endpoint_name = None

no stored variable or alias endpoint_name
no stored variable or alias model_package_arn
no stored variable or alias deployment_config
ERROR: No deployment configuration found.
Please set endpoint_name manually or run notebook 06 first.


In [3]:
# Display what will be deleted
if endpoint_name:
    print("Resources to be deleted:")
    print(f"  Endpoint: {endpoint_name}")
    if 'deployment_config' in dir():
        print(f"  Model S3 URI: {deployment_config.get('model_s3_uri')}")
        print(f"  Model Package ARN: {deployment_config.get('model_package_arn')}")
else:
    print("No endpoint configured for deletion.")

No endpoint configured for deletion.


## 3. Delete Endpoint

Delete the SageMaker endpoint to stop inference and charges.

**Note:** This is the most critical step to stop ongoing charges.

In [4]:
# Check if endpoint exists
if endpoint_name:
    try:
        endpoint_desc = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
        print(f"Endpoint found: {endpoint_name}")
        print(f"Status: {endpoint_desc['EndpointStatus']}")
        print(f"Created: {endpoint_desc['CreationTime']}")
    except sagemaker_client.exceptions.ClientError as e:
        if 'Could not find endpoint' in str(e):
            print(f"Endpoint not found: {endpoint_name}")
            endpoint_name = None
        else:
            raise e
else:
    print("No endpoint name specified.")

No endpoint name specified.


In [5]:
# Delete the endpoint
if endpoint_name:
    try:
        sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
        print(f"Endpoint deleted: {endpoint_name}")
    except Exception as e:
        print(f"Error deleting endpoint: {e}")
else:
    print("No endpoint to delete.")

No endpoint to delete.


## 4. Delete Model Resources

Delete endpoint configuration and SageMaker model.

In [6]:
# Delete endpoint configuration
if endpoint_name:
    endpoint_config_name = endpoint_name
    
    try:
        sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
        print(f"Endpoint config deleted: {endpoint_config_name}")
    except sagemaker_client.exceptions.ClientError as e:
        if 'Could not find endpoint configuration' in str(e):
            print(f"Endpoint config not found: {endpoint_config_name}")
        else:
            print(f"Error: {e}")
else:
    print("No endpoint config to delete.")

No endpoint config to delete.


In [7]:
# List and delete associated models
if 'deployment_config' in dir():
    model_name_prefix = 'cms-anomaly-model'
    
    try:
        models = sagemaker_client.list_models(
            NameContains=model_name_prefix,
            MaxResults=10
        )
        
        if models['Models']:
            for model in models['Models']:
                model_name = model['ModelName']
                try:
                    sagemaker_client.delete_model(ModelName=model_name)
                    print(f"Model deleted: {model_name}")
                except Exception as e:
                    print(f"Error: {e}")
        else:
            print("No models found.")
    except Exception as e:
        print(f"Error: {e}")
else:
    print("No deployment config available.")

No deployment config available.


## 5. Delete Model Registry Entries

Optionally delete model packages from the Model Registry.

**Note:** Only delete if you no longer need version history.

In [8]:
# List model packages in the group
model_package_group_name = 'cms-anomaly-detection-models'

try:
    response = sagemaker_client.list_model_packages(
        ModelPackageGroupName=model_package_group_name,
        MaxResults=10
    )
    
    if response['ModelPackageSummaryList']:
        print(f"Found {len(response['ModelPackageSummaryList'])} model package(s)")
        for pkg in response['ModelPackageSummaryList']:
            print(f"  {pkg['ModelPackageArn']} - {pkg['ModelPackageStatus']}")
    else:
        print("No model packages found.")
except sagemaker_client.exceptions.ClientError as e:
    if 'does not exist' in str(e):
        print(f"Model package group not found: {model_package_group_name}")
    else:
        print(f"Error: {e}")

Found 1 model package(s)
  arn:aws:sagemaker:us-east-1:864106638709:model-package/cms-anomaly-detection-models/1 - Completed


In [9]:
# Delete model packages
# WARNING: This deletes version history from Model Registry

DELETE_MODEL_PACKAGES = True  # Set to True to enable deletion

if DELETE_MODEL_PACKAGES:
    try:
        response = sagemaker_client.list_model_packages(
            ModelPackageGroupName=model_package_group_name
        )
        
        for pkg in response['ModelPackageSummaryList']:
            model_package_arn = pkg['ModelPackageArn']
            try:
                sagemaker_client.delete_model_package(ModelPackageName=model_package_arn)
                print(f"Model package deleted: {model_package_arn}")
            except Exception as e:
                print(f"Error: {e}")
        
        try:
            sagemaker_client.delete_model_package_group(
                ModelPackageGroupName=model_package_group_name
            )
            print(f"Model package group deleted: {model_package_group_name}")
        except Exception as e:
            print(f"Error: {e}")
            
    except Exception as e:
        print(f"Error: {e}")
else:
    print("Model package deletion skipped")

Model package deleted: arn:aws:sagemaker:us-east-1:864106638709:model-package/cms-anomaly-detection-models/1
Model package group deleted: cms-anomaly-detection-models


## 6. Delete S3 Artifacts

In [10]:
# List S3 objects in the project prefix
s3_prefix = 'cms-anomaly-detection'

try:
    response = s3_client.list_objects_v2(
        Bucket=bucket,
        Prefix=s3_prefix,
        MaxKeys=20
    )
    
    if 'Contents' in response:
        total_size = sum(obj['Size'] for obj in response['Contents'])
        print(f"Found {len(response['Contents'])} objects ({total_size / 1024 / 1024:.2f} MB)")
        for obj in response['Contents'][:5]:
            print(f"  {obj['Key']} ({obj['Size'] / 1024:.1f} KB)")
        if len(response['Contents']) > 5:
            print(f"  ... and {len(response['Contents']) - 5} more")
    else:
        print(f"No objects found in s3://{bucket}/{s3_prefix}")
except Exception as e:
    print(f"Error: {e}")

Found 20 objects (2.75 MB)
  cms-anomaly-detection-featurestore/payments/864106638709/sagemaker/us-east-1/offline-store/cms-payments-fg-07-21-40-34-1770500435/data/year=2026/month=02/day=07/hour=21/20260207T214035Z_0E9UXBDl3KDkllCB.parquet (140.7 KB)
  cms-anomaly-detection-featurestore/payments/864106638709/sagemaker/us-east-1/offline-store/cms-payments-fg-07-21-40-34-1770500435/data/year=2026/month=02/day=07/hour=21/20260207T214035Z_1REtBaMZ0RAQOt2P.parquet (140.8 KB)
  cms-anomaly-detection-featurestore/payments/864106638709/sagemaker/us-east-1/offline-store/cms-payments-fg-07-21-40-34-1770500435/data/year=2026/month=02/day=07/hour=21/20260207T214035Z_1VNoZXyT5fah8Uqq.parquet (147.7 KB)
  cms-anomaly-detection-featurestore/payments/864106638709/sagemaker/us-east-1/offline-store/cms-payments-fg-07-21-40-34-1770500435/data/year=2026/month=02/day=07/hour=21/20260207T214035Z_1piVnuEQSMqzKkLp.parquet (139.2 KB)
  cms-anomaly-detection-featurestore/payments/864106638709/sagemaker/us-east-

In [11]:
# Delete S3 objects
# WARNING: This permanently deletes model artifacts and data

DELETE_S3_ARTIFACTS = False  # Set to True to enable deletion

if DELETE_S3_ARTIFACTS:
    try:
        paginator = s3_client.get_paginator('list_objects_v2')
        pages = paginator.paginate(Bucket=bucket, Prefix=s3_prefix)
        
        deleted_count = 0
        for page in pages:
            if 'Contents' in page:
                for obj in page['Contents']:
                    s3_client.delete_object(Bucket=bucket, Key=obj['Key'])
                    deleted_count += 1
        
        print(f"Deleted {deleted_count} objects from s3://{bucket}/{s3_prefix}")
        
    except Exception as e:
        print(f"Error: {e}")
else:
    print("S3 artifact deletion skipped")

S3 artifact deletion skipped


## 7. Cleanup Summary

Display summary of cleanup actions performed.

In [12]:
# Create cleanup summary
print("CLEANUP SUMMARY")
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Region: {region}")
print(f"\nEndpoint deleted: {endpoint_name if endpoint_name else 'N/A'}")
print(f"Model Registry packages deleted: {DELETE_MODEL_PACKAGES}")
print(f"S3 artifacts deleted: {DELETE_S3_ARTIFACTS}")

CLEANUP SUMMARY
Date: 2026-02-15 05:09:26
Region: us-east-1

Endpoint deleted: N/A
Model Registry packages deleted: True
S3 artifacts deleted: False


In [13]:
# Verify endpoint is deleted
if endpoint_name:
    try:
        sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
        print(f"Endpoint still exists (may be deleting): {endpoint_name}")
    except sagemaker_client.exceptions.ClientError:
        print(f"Endpoint deletion confirmed")
else:
    print("No endpoint to verify.")

No endpoint to verify.
